![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "Dados/UCI HAR Dataset/features.txt"
filename_labels = "Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, usecols=[1], names = ['colunas'], sep= ' ')
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace = True, header=None)
X_train.columns = features['colunas'].tolist()
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None)
X_test.columns = features['colunas'].tolist()
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [2]:
%%time
clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(X_train, y_train)
print(f"R_quadrado na base de treino: {clf.score(X_train, y_train)*100:.2f}")
print(f"R_quadrado na base de teste: {clf.score(X_test, y_test)*100:.2f}")

R_quadrado na base de treino: 97.58
R_quadrado na base de teste: 87.92
CPU times: total: 6.81 s
Wall time: 6.95 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [3]:
%%time
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(pc_treino, y_train)
print(f"R_quadrado na base de treino: {clf.score(pc_treino, y_train)*100:.2f}")
print(f"R_quadrado na base de teste: {clf.score(pc_teste, y_test)*100:.2f}")

R_quadrado na base de treino: 49.97
R_quadrado na base de teste: 45.71
CPU times: total: 1.28 s
Wall time: 478 ms


#### Avalie a acurácia nas bases de treinamento e teste
- Houve uma queda significativa na acurácia, caindo tanto em treinamento quanto em teste quase 50%.

#### Avalie o tempo de processamento
- Houve melhora significativa com relação ao tempo de processamento: **CPU times** representando **18,80%** e **Wall time** representando **6,88%**.

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [4]:
# Criando uma função para as 'n' quantidades de componentes
def clf_n(componente):
    prcomp = PCA(n_components= componente).fit(X_train)

    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(pc_treino, y_train)
    print(f"Número de componentes: {componente}")
    print(f"R_quadrado na base de treino: {clf.score(pc_treino, y_train)*100:.2f}")
    print(f"R_quadrado na base de teste: {clf.score(pc_teste, y_test)*100:.2f}")

In [5]:
%%time
clf_n(1)

Número de componentes: 1
R_quadrado na base de treino: 49.97
R_quadrado na base de teste: 45.71
CPU times: total: 1.53 s
Wall time: 453 ms


In [6]:
%%time
clf_n(2)

Número de componentes: 2
R_quadrado na base de treino: 61.28
R_quadrado na base de teste: 58.47
CPU times: total: 1.12 s
Wall time: 352 ms


In [7]:
%%time
clf_n(5)

Número de componentes: 5
R_quadrado na base de treino: 84.60
R_quadrado na base de teste: 78.86
CPU times: total: 1.31 s
Wall time: 436 ms


In [8]:
%%time
clf_n(10)

Número de componentes: 10
R_quadrado na base de treino: 89.35
R_quadrado na base de teste: 82.29
CPU times: total: 1.55 s
Wall time: 552 ms


In [9]:
%%time
clf_n(50)

Número de componentes: 50
R_quadrado na base de treino: 92.00
R_quadrado na base de teste: 82.32
CPU times: total: 2.95 s
Wall time: 1.41 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

#### O que aconteceu com a acurácia?
- A acurácia apresenta valor crescente a medida que aumentamos a quantidade de componentes, contudo desacelerando o incremente da acurácia com o aumento da quantidade de componentes. 

#### O que aconteceu com o tempo de processamento?
- O tempo de processamento é **menor** para a quantidade de componentes igual a **'2'** e é **maior** para a quantidade de componentes igual a **'50'**. Contudo, ainda é bem inferior aos tempos obtidos no primeiro modelo, obtendo em relação a este um **CPU times** representando **43,32%** e um **Wall time** representando **20,29%**.